#Sentiment Analysis

###Importing

In [0]:
from collections import Counter
import numpy as np




###Data


In [17]:
# !unzip archive.zip
# %cd archive
#%cd data


/content/archive/data/data


In [0]:
positive_counter = Counter()
negative_counter = Counter()
total_counts = Counter()


data/


##Curating Dataset

In [0]:
def print_review_and_label(i):
  print(labels[i] + "\t:\t" + reviews[i][:80] + "...")
  
g = open('reviews.txt','r')
reviews = list(map(lambda x:x[:-1], g.readlines()))
g.close()

g = open('labels.txt','r')
labels = list(map(lambda x:x[:-1], g.readlines()))
g.close()


In [0]:
print_review_and_label(1)
print_review_and_label(2)
print_review_and_label(3)
print_review_and_label(4)
print_review_and_label(5)
print_review_and_label(6)

In [0]:
for i in range(len(reviews)):
  if(labels[i] == 'positive'):
    for word in reviews[i].split(" "):
      positive_counter[word] += 1
      total_counts[word] += 1
  else:
    for word in reviews[i].split(" "):
      negative_counter[word] += 1
      total_counts[word] += 1

##Create Input/Output Data

In [40]:
vocab = set(total_counts.keys())
vocab_size = len(vocab)
print(vocab_size)

74074


In [41]:
layer_0 = np.zeros((1,vocab_size))
layer_0

array([[0., 0., 0., ..., 0., 0., 0.]])

In [0]:
word2index = {}

for i,word in enumerate(vocab):
  word2index[word] = i


word2index


In [0]:
layer_0.shape

(1, 74074)

In [48]:
labels[1]

'negative'

In [0]:
def update_input_layer(review):
  
  global layer_0
  
  #clear out all previous state, reset the layer to all 0s
  layer_0 *= 0
  for word in review.split(" "):
    layer_0[0][word2index[word]] += 1
    
update_input_layer(reviews[0])    

In [0]:
def get_target_for_label(label):
  if(label == 'positive'):
    return 1
  else:
    return 0
  

In [0]:
for i in range(0,100):
  print(get_target_for_label(labels[i]))

##Create Neural Network

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [0]:
class neuralnet(nn.Module):
  
  
  def __init__(self,input_dim,hidden_dim,output_dim):
    super(neuralnet, self).__init__()
    
    # Assign a seed to our random number generator to ensure we get reproducable results during development
    np.random.seed(1)
    
    # process the reviews and their associated labels so that everything is ready for training
    self.pre_process_data(reviews, labels)
    
    # Build the network to have the number of hidden nodes and the learning rate that were passed into this initializer. Make the same number of input nodes as
    # there are vocabulary words and create a single output node.
      self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)
    
    # Linear Function
    self.f1 = nn.Linear(input_dim, hidden_dim)
    
    # Non-Linear Function
    self.sigmoid = nn.Sigmoid()
    
    # Linear Function (readout)
    self.f2 = nn.Linear(hidden_dim, output_dim)
    
  
    
  def pre_process_data(self, reviews, labels):
    
    review_vocab = set()            # set does not contain dupliactes
    for review in reviews:
      for word in reviews.split(" "):
        review_vocab.add(word)
    self.review_vocab = list(review_vocab)       # coverting to list for easy iteration
    
    
    label_vocab = set()
    for label i labels:
      label_vocab.add(label)
    self.label_vocab = list(label_vocab)
    
   
    self.review_vocab_size = len(review_vocab)
    self.review_vocab_size = len(label_vocab)

    self.word2index = {}
    for i, word in enumerate(self.review_vocab):       # enumerate allows us to loop over something and have an automatic counter.
      self.word2index[word] = i

    self.word2index = {}
    for i, label in enumerate(self.label_vocab):       # enumerate allows us to loop over something and have an automatic counter.
      self.word2index[word] = i
      
      
      
      
  def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
    # Set values
    self.input_nodes = review_vocab_size
    self.hidden_nodes = hidden_nodes
    self.output_nodes = review_vocab_size
      
    # Set learning rate
    self.learning_rate = learning_rate
    
    # Initialize weights
    
    # Weights between input layer and hidden layer
    self.weights_0_1 = np.zeros((self.input_nodes, self.hidden_nodes))
    
    # Weights between hidden layer and output layer
    self.weights_1_2 = np.random.normal(0.0, self.output_nodes**-0.5, (self.hidden_nodes, self.output_nodes))   # syntax  numpy.random.normal(loc=0.0, scale=1.0, size=None)
    
    # The input layer, 2 fim matrix, 1 x hidden inputs
    self.layer_1 = np.zeros((1, hidden_nodes)
                            
                            
  def sigmoid(self,x):
    return 1 / (1 + np.exp(-x))
                 
  def sigmoid_output_2_derivative(self,output):
    return output * (1 - output)                        
      
                            
  def get_target_for_label(label):
    if(label == 'positive'):
      return 1
    else:
      return 0
  
  def train(self, trainig_reviews_raw, training_labels):
    training_reviews = list()
    for review in training_reviews_raw:
      indices = set()
            for word in review.split(" "):
                if(word in self.word2index.keys()):
                    indices.add(self.word2index[word])
            training_reviews.append(list(indices))
    
   
                  
                            
    
  
      